Generative QA v1 following [this](https://haystack.deepset.ai/tutorials/07_rag_generator). This is intended to test the possibility of building the database purely for Pale.

In [1]:
import os
print(os.getcwd())
os.chdir('./drive/MyDrive/pale-companion-files/')

/content


In [2]:
os.getcwd()

'/content/drive/MyDrive/pale-companion-files'

In [4]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-ttvfc1v0/farm-haystack_0373c8585c064b29bbad6099dc46811b
  Resolved https://github.com/deepset-ai/haystack.git to commit 75ef9596780a18b4c5ec83777aeede1aef53be89
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 5.6 MB/s eta 0:00:0

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-ttvfc1v0/farm-haystack_0373c8585c064b29bbad6099dc46811b


In [15]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)


In [6]:
# Import some files
import pickle
with open('./chapter_fmt_list.pkl','rb') as f:
    chapters = pickle.load(f)

# Temporary
# chapters = chapters[:100]

In [7]:
from haystack import Document
chapter_documents = [Document.from_dict(d) for d in chapters]
len(chapter_documents)

307

In [8]:
from haystack.nodes import PreProcessor

word_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=20,
    progress_bar=True, 
    add_page_number=True
)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable  HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
docs_word = word_preprocessor.process(chapter_documents)
len(docs_word)

Preprocessing:   0%|          | 0/307 [00:00<?, ?docs/s]

53227

In [10]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever


# Initialize FAISS document store.
# Set `return_embedding` to `True`, so generator doesn't have to perform re-embedding
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Initialize DPR Retriever to encode documents, encode question and query documents
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=False,
)

# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=2,
    max_length=200,
    min_length=2,
    embed_title=False,
    num_beams=3,
)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model:Auto-detected model language: english


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


/usr/local/lib/python3.8/dist-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RagTokenForGeneration were not initialized from the model checkpoint at facebook/rag-token-nq and are newly initialized: ['rag.generator.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [11]:
# Delete existing documents in documents store
document_store.delete_documents()

# Batch the write / embeddings update because my machine doesn't have much memory
# chunks = [docs_word[x:x+100] for x in range(0, len(docs_word), 100)]

# Write documents to document store
document_store.write_documents(docs_word)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)


INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manu

Create embeddings:   0%|          | 0/3232 [00:00<?, ? Docs/s]

In [12]:
document_store.save("pale_gen_qa_v1.faiss") # Hopefully we can load this on my local machine?


In [13]:
QUESTIONS = [
    "Who is Avery?",
    "Where does Verona live?",
    "What is Toadswallow?",
    "How many Carmine conspirators are there?",
    "What does Alpeona do?",
    "Why is Gilkey ostracized?",
    "What is Snowdrop?",
    "Why is Maricaca scared of?",
    "What are the opinions of Charles?",
    "How many others live in Kennet?",
    "Why was Seth Forsaken?",
    "Is Alexander dead?",
    "Where is Bristow?",
    "What is the Wolf?",
    "What is the Red Heron?",
    "Can we talk about the girls?",
    "Who has a crush on Avery?",
    "What are the most powerful Others?",
    "What are the boons for Sootsleeves path?"
]


In [16]:
# Or alternatively use the Pipeline class
from haystack.pipelines import GenerativeQAPipeline
from haystack.utils import print_answers

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
for question in QUESTIONS:
    res = pipe.run(query=question, params={"Generator": {"top_k": 2}, "Retriever": {"top_k": 10}})
    print_answers(res, details="all")



Query: Who is Avery?
Answers:
[   <Answer {'answer': ',', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['e0a2a8d4ba6bb7fc5f066113398560c4', '5e48014db8dd94b96063e24480fabe75', '5a526e65d774f66d5970a5043028a8fb', '8c5f83007da1ea2312745f02c19ff894', 'a6fb08e5453396c72da13c64aef7ff90', 'f92e7af498f88a5ea51c1277c7b8f7ff', '82e5990f444e658ed8930c5db98eaa1', 'b0c79c04589ceac29ca85d8aaccedb6c', '482ba9330bf9e70ab4fe7e38d9900b1a', '5686fd9f6067c0e621311e7cda404770'], 'meta': {'doc_scores': [0.676339818149354, 0.6761715320640017, 0.6733548696066828, 0.6723385062584493, 0.6716912579736398, 0.6714387398891116, 0.6712842790682889, 0.6712673763545767, 0.6709374321698239, 0.6709084594833653], 'content': ['Booker is a treasure, I’m immensely proud.  But he’s had wake up calls about what the world had in store for him.  A tough first semester.”  “I stand by the idea that if everyone in the world was more like Booker, th